### 라이브러리 Import

In [1]:
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from typing import Optional, Any

from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

from dotenv import load_dotenv
load_dotenv()

True

### 시스템 설정

In [2]:
# Chroma
CHROMA_DB_PATH = "./chroma_db" 

# Neo4j
USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")
URL = os.environ.get("URL")

# OpenAI
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)

### RAG

In [ ]:
# RAG 프롬프트

RAG_PROMPT_TEMPLATE = """
당신은 Bella Roma 레스토랑 데이터 전문가를 위한 유용한 AI 어시스턴트입니다.
제공된 Context (검색된 문서)만을 사용하여 사용자의 질문에 답변하십시오.
만약 Context에 답변할 수 있는 정보가 없다면, '제공된 정보로는 답변할 수 없습니다.'라고 명확하게 언급하세요.

[질문]
{question}

[Context (검색된 문서)]
{context}
"""

In [4]:
def initialize_chroma_db(embedding_function, collection_name="unified_data"):
    """ ChromaDB 벡터 스토어를 초기화합니다. (가정: 데이터는 이미 로드되어 있습니다) """
    try:
        # 이미 데이터가 존재하는 컬렉션에 연결합니다.
        vector_db = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embedding_function,
            collection_name=collection_name
        )
        
        collection = getattr(vector_db, "_collection", None)
        count = collection.count()
        print(f"연결된 컬렉션(unified_data) 문서 개수: {count}")
        print()
        
        return vector_db
    except Exception as e:
        print(f"ChromaDB 초기화 오류: {e}")
        return None

In [52]:
def rag_pipeline(question: str, llm: ChatOpenAI) -> str:
    """
    Vector DB (ChromaDB)를 사용하여 비정형 데이터 기반 RAG 파이프라인을 LCEL로 실행합니다.
    """
    # 1. Vector DB 초기화 및 임베딩 함수 설정
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vector_db = initialize_chroma_db(embeddings, collection_name="unified_data")
    
    if vector_db is None:
        return "ERROR: Vector DB를 초기화할 수 없습니다."

    # 2. Retriever 및 Prompt Template 정의
    retriever = vector_db.as_retriever(search_kwargs={"k": 5})
    prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)
    
    docs = retriever.invoke(question)
        
    # 검색된 문서 내용을 하나의 문자열로 결합
    context = "\n---\n".join([doc.page_content for doc in docs])

    # 3. LCEL 체인 구성 (RunnablePassthrough 사용)
    # 체인 흐름: 
    # {context: 검색(retriever), question: 질문 원문(RunnablePassthrough)} 
    # -> Prompt -> LLM -> 결과 파싱
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    # 4. 질문 실행 및 결과 생성
    try:
        # invoke 메서드는 LCEL 체인을 실행하며, 입력은 question 문자열입니다.
        result = rag_chain.invoke(question)
        return context, result
    except Exception as e:
        return f"ERROR: RAG 파이프라인 실행 오류: {e}"

# 예시 사용법 (실제 실행 시 주석 해제)
question = "Bella Roma 레스토랑의 영업시간을 알려주세요"
# question = "Bella Roma 레스토랑의 영업시간을 알려주세요."
# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0) # llm 객체 초기화 필요
context, answer = rag_pipeline(question, llm)
print(f"RAG Context: {context}")
print()
print(f"RAG 답변: {answer}")

RAG Context: # Bella Roma 직원 정보

## 팀원 소개

이 문서에서는 Bella Roma 레스토랑을 이끌어가는 주요 직원들의 정보를 소개합니다.

---

### 김철수 (주방장, Head Chef)

-   **역할**: 주방장 (Head Chef)
-   **경력**: 15년
-   **근무 요일**: 월요일 ~ 금요일
-   **소개**: 15년 경력의 베테랑 주방장인 김철수 셰프는 Bella Roma의 주방을 총괄하며, 모든 메뉴의 맛과 품질을 책임지고 있습니다.

---

### 이영희 (매니저)

-   **역할**: 매니저
-   **경력**: 10년
-   **근무 요일**: 월요일 ~ 토요일
-   **소개**: 10년 경력의 이영희 매니저는 레스토랑의 전반적인 운영, 고객 응대, 그리고 직원 관리를 담당하며 고객들이 최상의 경험을 할 수 있도록 돕습니다.

---

### 박민수 (서버)

-   **역할**: 서버
-   **경력**: 2년
-   **근무 요일**: 화요일 ~ 일요일
-   **소개**: 2년 경력의 박민수 서버는 밝은 에너지로 고객들의 식사를 돕는 홀 서빙 전문가입니다. 메뉴 추천과 고객의 편의를 책임집니다.

---
---
# 식당 정보: Bella Roma

식당은 서울 신촌에 위치한 이탈리안 레스토랑 'Bella Roma'입니다. 상세 정보는 다음과 같습니다.

-   **이름**: Bella Roma
-   **카테고리**: 이탈리안 레스토랑
-   **위치**: 서울특별시 서대문구 신촌로 123
-   **전화번호**: 02-123-4567
-   **영업시간**: 매일 11:00부터 22:00까지이며, 주말에는 23:00까지 연장 운영합니다.
-   **좌석 수**: 60석
-   **예약**: 전화 및 온라인 예약이 가능합니다.
-   **결제 수단**: 현금, 카드, 그리고 다양한 간편결제를 지원합니다.
---
# Bella Roma 메뉴 정보

## 메뉴 상세 설명

이 문서에서는 B

### Graph RAG

In [6]:
def initialize_neo4j_graph() -> Neo4jGraph:
    """ Neo4jGraph 객체를 초기화합니다. """
    # 실제 환경에서는 NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD를 사용해야 합니다.
    try:
        # 임시 URL/인증 정보를 가정하거나 환경 변수에서 로드합니다.
        graph = Neo4jGraph(
            url="bolt://52.3.233.24:7687", # 실제 URL로 변경 필요
            username="neo4j", 
            password="canvases-return-armaments"
        )
        # 스키마가 미리 로드되어 있다고 가정
        return graph
    except Exception as e:
        print(f"Neo4j 초기화 오류: {e}")
        return None

In [8]:
def graph_rag_pipeline(question: str, llm: ChatOpenAI) -> str:
    """
    Graph DB (Neo4j)를 사용하여 스키마 기반 Cypher 쿼리 생성 파이프라인을 실행합니다.
    """
    # 1. Neo4j Graph 초기화
    graph = initialize_neo4j_graph()
    if graph is None:
        return "ERROR: Neo4j Graph를 초기화할 수 없습니다."
    
    # 2. GraphCypherQAChain 생성
    # LLM에 스키마를 전달하여 Cypher 쿼리 생성 및 실행 후 답변 생성
    qa_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=False,
        allow_dangerous_requests=True,
        # LLM이 쿼리를 생성하고 결과를 자연어로 해석할 때 스키마가 Context로 사용됨
    )
    
    # 3. 질문 실행 및 결과 생성
    try:
        result = qa_chain.invoke({"query": question})
        return result["result"]
    except Exception as e:
        return f"ERROR: GRAPH RAG 파이프라인 실행 오류: {e}"

# # 예시 사용법 (실제 실행 시 주석 해제)
question = "마르게리타 피자는 어떤 재료를 포함하며, 유제품 알러지를 유발하나요?"
answer = graph_rag_pipeline(question, llm)
print(f"GRAPH RAG 답변: {answer}")

C:\Users\faeqs\AppData\Local\Temp\ipykernel_14944\2560474760.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


GRAPH RAG 답변: 마르게리타 피자는 밀가루, 토마토, 치즈, 바질을 포함하며, 유제품 알러지를 유발합니다.


### Hybrid (RAG & Graph RAG)

In [ ]:
HYBRID_QA_TEMPLATE = """
당신은 Bella Roma 레스토랑 데이터 분석 전문가입니다.
아래에 제공된 두 가지 검색 결과 (Graph DB Context 및 Vector DB Context)를 모두 활용하여 사용자의 질문에 대한 가장 정확하고 포괄적인 답변을 한국어로 생성하십시오.

[질문]
{question}

--- Graph DB Context ---
{graph_context}
---------------------------------------

--- Vector DB Context ---
{vector_context}
-----------------------------------------

# 응답 가이드라인:
- 두 Context를 종합하여 답변하며, 충돌하는 정보가 있을 경우 Graph DB의 사실 정보를 우선하세요.
- 답변은 전문적이고 유익한 톤을 사용하세요.
- 두 Context 모두에 답변할 정보가 없는 경우, '제공된 정보로는 답변을 통합할 수 없습니다.'라고 명확히 언급하세요.
"""

In [10]:
HYBRID_QA_PROMPT = ChatPromptTemplate.from_template(HYBRID_QA_TEMPLATE)

In [ ]:
def initialize_chroma_db(embedding_function: OpenAIEmbeddings, collection_name: str) -> Optional[Chroma]:
    """ ChromaDB 벡터 스토어를 초기화합니다. """
    try:
        vector_db = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embedding_function,
            collection_name=collection_name
        )
        # print(f"연결된 컬렉션({collection_name}) 문서 개수: {getattr(vector_db, '_collection').count()}")
        return vector_db
    except Exception as e:
        print(f"ChromaDB 초기화 오류: {e}")
        return None

In [12]:
def initialize_neo4j_graph() -> Optional[Neo4jGraph]:
    """ Neo4jGraph 객체를 초기화합니다. """
    try:
        graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)
        return graph
    except Exception as e:
        print(f"Neo4j 초기화 오류: {e}")
        return None


In [45]:
def fetch_vector_context(question: str, llm: ChatOpenAI) -> str:
    """ Vector DB에서 관련 문서의 Context를 검색합니다. """
    try:
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        vector_db = initialize_chroma_db(embeddings, collection_name="unified_data")
        
        if vector_db is None:
            return "ERROR: Vector DB 초기화 실패"
        
        # 5개 문서를 검색하여 텍스트 Context로 변환
        retriever = vector_db.as_retriever(search_kwargs={"k": 5})
        prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)
        
        rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
        
        
        docs = retriever.invoke(question)
        
        # 검색된 문서 내용을 하나의 문자열로 결합
        context = "\n---\n".join([doc.page_content for doc in docs])
        
        result = rag_chain.invoke(question)
        
        return context, result
        
    except Exception as e:
        return f"ERROR: Vector DB 검색 오류: {e}"

---

In [ ]:
# Graph RAG - Cypher 생성을 위한 레스토랑 데이터베이스 특화 프롬프트

CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to question a restaurant and customer review graph database (focused on Bella Roma).
Instructions:
- Use only the provided node labels, relationship types, and properties in the schema.
- Do not use any relationship types or properties not specified in the schema.
- Focus on extracting meaningful insights from restaurant, menu, purchase, and review data.

Schema:
{schema}

Note: 
- Provide only the Cypher statement.
- Do not include explanations or apologies.
- Generate precise, relevant Cypher queries.

Examples:
# 특정 직원의 근무일과 역할 조회
MATCH (e:Employee)
WHERE e.name = '김철수'
RETURN e.name, e.role, e.workingDays

# 특정 고객의 총 구매 횟수와 최고 구매액 조회
MATCH (c:Customer)<-[:BY_CUSTOMER]-(p:Purchase)
WHERE c.name = '사용자11'
RETURN c.name, COUNT(p) AS totalPurchases, MAX(p.totalPrice) AS maxPurchaseAmount

# 특정 메뉴의 재료와 유발 알러지 조회
MATCH (m:MenuItem {{name: '까르보나라 파스타'}})-[:CONTAINS_INGREDIENT]->(i:Ingredient)
OPTIONAL MATCH (m)-[:MAY_TRIGGER_ALLERGY]->(a:Allergy)
RETURN m.name, COLLECT(i.name) AS Ingredients, COLLECT(a.name) AS Allergies

# 가장 많이 팔린 메뉴 항목과 판매 수량 조회
MATCH (m:MenuItem)<-[:FOR_MENU_ITEM]-(p:Purchase)
RETURN m.name, SUM(p.quantity) AS TotalQuantity
ORDER BY TotalQuantity DESC
LIMIT 5

# 4점 이상 긍정적 리뷰를 가장 많이 남긴 고객 조회
MATCH (c:Customer)<-[:WRITTEN_BY]-(r:CustomerReview)
WHERE r.rating >= 4
RETURN c.name, COUNT(r) AS PositiveReviewCount
ORDER BY PositiveReviewCount DESC
LIMIT 10

The question is:
{question}"""

# 결과 처리를 위한 레스토랑 QA 프롬프트
QA_TEMPLATE = """
당신은 Bella Roma 레스토랑 데이터 분석 전문가로, 메뉴, 고객 행동 및 운영 정보에 대한 명확하고 간결한 정보를 한국어로 제공합니다.

[질문]
{question}

[검색 결과 (Cypher 쿼리 실행 결과)]
{context}

# 응답 가이드라인:
- 검색 결과에서 핵심 정보를 요약하세요
- 레스토랑 데이터에 대한 명확하고 객관적인 개요를 제공하세요
- 전문적이고 유익한 톤을 사용하세요
- 고객 행동, 인기 메뉴, 운영 효율성 측면에서 중요한 패턴이나 트렌드를 강조하세요
- 맥락이 불충분한 경우 더 많은 정보가 필요하다고 명확히 언급하세요
- 추측이나 개인적인 해석은 피하세요

# 응답 형식:
- 간략한 발견 요약으로 시작하세요
- 여러 항목이 발견된 경우 (예: 메뉴 목록, 고객 목록) 간결한 개요를 제공하세요
- 가독성을 위해 글머리 기호나 짧은 단락을 사용하세요
- 가격, 수량, 평점, 날짜와 같은 관련 수치 정보를 이해하기 쉬운 언어로 번역하세요

# 예시 응답 구조:
"분석 결과, [주요 발견 요약]

주요 특징/상세 정보:
- [첫 번째 중요 인사이트]
- [두 번째 중요 인사이트]

추가 정보: [필요한 경우 추가 설명]"
"""

from langchain_core.prompts.prompt import PromptTemplate

# PromptTemplate 객체 생성
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE)

QA_PROMPT = PromptTemplate(
    input_variables=["question", "context"], 
    template=QA_TEMPLATE)

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

graph = initialize_neo4j_graph()
# GraphCypherQAChain 생성 및 실행
qa_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    qa_prompt=QA_PROMPT,
    input_key="question",  
    output_key="result",
    # graph.schema를 cypher_prompt에 schema로 전달
    prompt_kwargs={"schema": graph.schema}
)

In [33]:
question = "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴와 메뉴 카테고리는 무엇인가요?"

In [34]:
result = qa_chain.invoke({"question": question})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant {name: 'Bella Roma'})<-[:REVIEWS_RESTAURANT]-(rev:CustomerReview)-[:WRITTEN_BY]->(c:Customer)
WHERE rev.rating >= 4
  AND rev.reviewDate >= date('2024-01-01') AND rev.reviewDate <= date('2024-06-30')
WITH COLLECT(DISTINCT c) AS customers, r
CALL {
  WITH customers, r
  UNWIND customers AS cust
  MATCH (cust)<-[:BY_CUSTOMER]-(p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem)-[:IN_MENU_CATEGORY]->(mc:MenuCategory), (p)-[:AT_RESTAURANT]->(r)
  RETURN m.name AS menuItem, mc.name AS menuCategory, SUM(p.quantity) AS qty
  ORDER BY qty DESC
  LIMIT 1
}
CALL {
  WITH customers, r
  UNWIND customers AS cust
  MATCH (cust)<-[:BY_CUSTOMER]-(p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem)-[:IN_MENU_CATEGORY]->(mc:MenuCategory), (p)-[:AT_RESTAURANT]->(r)
  RETURN mc.name AS topCategory, SUM(p.quantity) AS categoryQty
  ORDER BY categoryQty DESC
  LIMIT 1
}
RETURN menuItem AS TopMenuItem, menuCategory AS TopMenuItemCategory, qty 

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (customers, r) { ... }} {position: line: 5, column: 1, offset: 261} for query: "MATCH (r:Restaurant {name: 'Bella Roma'})<-[:REVIEWS_RESTAURANT]-(rev:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nWHERE rev.rating >= 4\n  AND rev.reviewDate >= date('2024-01-01') AND rev.reviewDate <= date('2024-06-30')\nWITH COLLECT(DISTINCT c) AS customers, r\nCALL {\n  WITH customers, r\n  UNWIND customers AS cust\n  MATCH (cust)<-[:BY_CUSTOMER]-(p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem)-[:IN_MENU_CATEGORY]->(mc:MenuCategory), (p)-[:AT_RESTAURANT]->(r)\n  RETURN m.name AS menuItem, mc.name AS menuCategory, SUM(p.quantity) AS qty\n  ORDER BY qty DESC\n  LIMIT 1\n}\nCALL {\n  WITH customers, r\

Full Context:
[{'TopMenuItem': '까르보나라 파스타', 'TopMenuItemCategory': '파스타', 'TopMenuItemQty': 139, 'TopCategory': '파스타', 'TopCategoryQty': 139}]

> Finished chain.


In [35]:
result

{'question': '2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴와 메뉴 카테고리는 무엇인가요?',
 'result': '분석 결과, 2024년 상반기(1~6월)에 긍정적 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴는 "까르보나라 파스타"이며, 같은 기간 최다 주문 카테고리는 "파스타"입니다. 집계된 주문 수는 각각 139건입니다.\n\n주요 특징/상세 정보:\n- 최다 주문 메뉴: 까르보나라 파스타 — 139건\n- 최다 주문 카테고리: 파스타 — 139건\n- 주목할 점: 제공된 결과에서 TopMenuItemQty(139)와 TopCategoryQty(139)가 동일하게 보고되어 있습니다. 이는 본 결과 집계에서 해당 메뉴의 주문 수가 그 카테고리의 집계 수와 일치함을 의미합니다(추가 데이터 없이 그 이유를 단정할 수는 없습니다).\n\n운영·고객 행동 관점에서 고려할 점(관찰 기반, 추가 검증 권장):\n- 고객 만족도가 높은 그룹에서 파스타, 특히 까르보나라의 선호도가 두드러집니다. 이는 메뉴 강조, 추천 상품 배치, 관련 재고·재료 확보 우선순위에 영향을 줄 수 있습니다.\n- 긍정 리뷰와 특정 메뉴의 높은 상관성은 마케팅(예: 리뷰 기반 추천) 또는 교차판매 기회로 연결할 수 있는 단서가 됩니다.\n- 주방 준비·재고 측면에서는 상위 인기 메뉴에 대한 안정적 공급·표준화가 운영 효율성 제고에 도움이 됩니다.\n\n필요한 추가 정보(맥락 보완 시 더 정확한 해석·조치 가능):\n- 동일 기간 내 긍정 리뷰(4점 이상) 전체 주문 건수 및 전체 긍정 리뷰 수(비율 산출용)\n- 다른 메뉴 및 카테고리의 순위별 집계(비중 파악)\n- 월별 분해(1~6월 추이), 방문 채널(배달/포장/매장), 단골 여부 등 세분화 데이터\n- 매출액·평균 결제액(해당 메뉴의 매출 기여도 분석용)\n- 프로모션 또는 메뉴 변경이 있었는지 여부(원인 분석용)\n\n추가 데이터 제공이 가능하면, 비중(점유율)·

In [36]:
def fetch_graph_context(question: str, llm: ChatOpenAI) -> str:
    """ Graph DB에서 Cypher 쿼리를 생성 및 실행하여 결과를 반환합니다. """
    graph = initialize_neo4j_graph()
    if graph is None:
        return "ERROR: Neo4j Graph 초기화 실패"
    
    # GraphCypherQAChain 생성 및 실행
    qa_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=False,
        allow_dangerous_requests=True,
        cypher_prompt=CYPHER_GENERATION_PROMPT,
        qa_prompt=QA_PROMPT,
        input_key="question",  
        output_key="result",
        # graph.schema를 cypher_prompt에 schema로 전달
        prompt_kwargs={"schema": graph.schema}
    )
    
    try:
        # LLM은 Cypher 쿼리 실행 결과를 바탕으로 자연어 Context를 생성하여 반환합니다.
        result = qa_chain.invoke({"question": question})

        return result["result"]
    
    except Exception as e:
        return f"ERROR: GRAPH RAG 실행 오류: {e}"

In [ ]:
# 하이브리드 파이프라인

def hybrid_rag_pipeline(question: str, llm: ChatOpenAI) -> str:
    """
    Graph DB (정형)와 Vector DB (비정형)의 Context를 결합하여 최종 답변을 생성합니다.
    """
    print("--- 1. Graph DB 컨텍스트 검색 시작 ---")
    # (a) Graph DB 검색 - 정형 데이터 및 관계 정보
    graph_context = fetch_graph_context(question, llm)
    print(f"Graph Context: {graph_context}...")

    print("--- 2. Vector DB 컨텍스트 검색 시작 ---")
    # (b) Vector DB 검색 - 비정형 텍스트 문서
    vector_context, _ = fetch_vector_context(question, llm)
    print(f"Vector Context: {vector_context}...")

    # 3. Context 결합 및 최종 답변 생성 (LCEL 사용)
    
    # Context를 결합하는 Runnable
    combined_context_runnable = RunnableLambda(
        lambda q: {
            "question": q, 
            "graph_context": graph_context, 
            "vector_context": vector_context
        }
    )
    
    # LCEL 체인 구성: 입력 -> Context 결합 -> Prompt -> LLM -> 결과 파싱
    hybrid_chain = (
        combined_context_runnable
        | HYBRID_QA_PROMPT
        | llm
        | StrOutputParser()
    )
    
    print("--- 3. 하이브리드 LLM 추론 시작 ---")
    try:
        final_answer = hybrid_chain.invoke(question)
        return final_answer
    except Exception as e:
        return f"ERROR: 하이브리드 LLM 추론 오류: {e}"

In [ ]:
# 실행 예시 (사용자가 직접 실행)
question_hybrid = "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴 카테고리는 무엇인가요?"

llm_instance = ChatOpenAI(model_name="gpt-5-mini", temperature=0) 
answer = hybrid_rag_pipeline(question_hybrid, llm_instance)
print("\n=== 최종 하이브리드 RAG 답변 ===")
print(answer)

--- 1. Graph DB 컨텍스트 검색 시작 ---
Graph Context: 분석 결과, 2024년 상반기(1~6월) 동안 평점 4점 이상(긍정적 리뷰)을 남긴 고객들이 가장 많이 주문한 메뉴 카테고리는 "디저트"이며, 총 주문 수량은 73건입니다.

주요 특징/상세 정보:
- 카테고리: 디저트
- 총 주문 수량: 73건 (쿼리 결과의 TotalQuantity 값)
- 분석 조건: 2024년 1월~6월, 고객 평점 ≥ 4점
- 제공된 결과가 단일 행(디저트만)으로 반환되어, 다른 카테고리와의 비교값은 결과에 포함되어 있지 않습니다

관찰 가능한 패턴/시사점:
- 긍정적 리뷰를 남긴 고객군에서는 디저트가 가장 많이 주문된 카테고리로 나타났습니다. 이는 해당 고객 세그먼트에서 디저트의 인기 또는 자주 포함되는 품목임을 시사합니다(단, 인과관계는 확인되지 않음).

추가로 확인이 필요하거나 도움이 되는 정보:
- TotalQuantity의 정확한 정의(아이템 수 vs. 주문 라인 수)
- 다른 메뉴 카테고리별 주문 수(비교용)
- 긍정적 리뷰를 남긴 고유 고객 수 및 이들의 평균 주문 건수/구매금액
- 메뉴별(품목별) 상세 주문량 및 매출 기여도
- 월별 추이(1~6월 중 특정 월에 집중된지 여부)

원하시면 위 추가 항목 중 하나를 기준으로 상세 분석(예: 품목별 상위 디저트 목록, 매출 기여도 비교, 월별 추이 등)을 실행해 드리겠습니다. 어떤 정보를 더 원하시나요?...
--- 2. Vector DB 컨텍스트 검색 시작 ---
Vector Context: 사용자 사용자43이 2024-04-30에 평점 4점으로 '가격은 조금 비싸지만 서비스는 만족스러워요.'라는 리뷰를 남겼습니다.
---
사용자 사용자34이 2024-10-26에 평점 4점으로 '직원분들이 친절해서 기분 좋게 식사했습니다.'라는 리뷰를 남겼습니다.
---
사용자 사용자4이 2024-09-22에 평점 3점으로 '가격은 조금 비싸지만 서비스는 만족스러워요.'라는 리뷰를 남겼습니다.
---
사용자